In [1]:
import re
import pandas as pd
import numpy as np

pd.set_option('display.max.rows', 2000)

paid_customer_pat = r'PAID\b.*?\bCUSTOMER'

paid_customer_compiler = re.compile(paid_customer_pat)

no_repair_keys = [
    'NO REPAIR', r'\bW[^\W_]{0,15}OUT\b', r'\b\"?NO"?\b[\W_]{0,15}FIX\b',
    r'CUSTOMER DOESN’T WANT TO \b(?:FIX|REPAIR|REPLACE)\b', 
    r'SUGGEST\b.*?\bFOR REPAIR', 'NOT TO REPAIR', 'RETURN DEVICE', 'NO ISSUE',
    r'\bCANCEL\b[^\W]ORDER\b', 'TRANSFER JOB', r'\bWON\'?T\b\s+BOOT\b', 'EXPENSIVE',
    r'\bCAN\'?T\b\s+FIX\b', 'VOID', 'NOT WORTH', 'IS WORKING', 'DAMAGED', 'COSTLY',
    'CANNOT FIX', 'RECYCLE', 'DIAGNOSTIC'
]

no_repair_pat = '|'.join(no_repair_keys)

no_repair_compiler = re.compile(no_repair_pat)

courtesy_keys = ['COURTESY', 'COMPLAIN', 'FREE', 'COMPLIMENTARY', 'WARRANTY']

courtesy_pat = '|'.join(courtesy_keys)

courtesy_compiler = re.compile(courtesy_pat)

no_tax_pat = r'CASH|CHECK'

no_tax_compiler = re.compile(no_tax_pat)

def paid_customers_fnc(cell):
    """
    Function references to Payment Method column.
    Capture scenarios of having paid customers in list objects and strings.
    Capture inclination of excluding payment method when no sale is made.
    """
    if isinstance(cell, list): # Search pattern in list objects
        return any(paid_customer_compiler.search(pym) for pym in cell)
    if isinstance(cell, str): # Search pattern in strings
        return bool(paid_customer_compiler.search(cell))
    if pd.isna(cell): # Consider absence of payment method as a no-sale
        return True
    return False # Return everything else as False
    
def n_price_coherence_fnc(cell):
    """
    Function references to Price column.
    Ensure service fees are consistent with the negative totals.
    """
    if isinstance(cell, list): # If list, return True if sum is negative
        return sum(cell) <= 0
    if isinstance(cell, float): # If float, return True if negative
        return cell <= 0
    if pd.isna(cell): # If value is null or missing, return True
        return True
    return False # Return everything else as False

def base_fnc(cell, compiler):
    """
    Function references to Service column for mask_no_repairs and mask_courtesies.
    Capture scenarios where no repair was requested or needed.
    Capture scenarios where the company gives a free repair or warranty repair.
    
    Function references to Payment Method column for mask_zero_tax.
    Capture scenarios where the payment method was cash, resulting in the zero tax.
    """
    if isinstance(cell, list): # Search pattern in list objects
        return any(compiler.search(key) for key in cell)
    if isinstance(cell, str): # Search pattern in strings
        return bool(compiler.search(cell))
    return False # Return everything else as False

def infer_payment_fnc(row):
    """ 
    Infer cash payment if...
    a) zero or null tax AND
    b) subtotal == total OR
    c) subtotal - discount == total
    
    Infer credit card payment if...
    a) tax AND
    b) subtotal + tax - discount == total
    """
    payment = row['Payment Method']
    tax = row['Tax']
    subtotal = row['Subtotal']
    discount = row['Discount']
    total = row['Total']

    if pd.notna(subtotal) and pd.notna(total):
        if (
            (pd.isna(tax) or tax == 0)
            and (
                (subtotal == total) 
                or (np.isclose(subtotal - discount, total))
            )
        ):
            payment = 'PAID CASH OR CHECK'
            return payment
        
        if (
            (pd.notna(tax) and tax > 0)
            and (
                (pd.notna(tax))
                and (np.isclose(subtotal + tax - discount, total))
            )
        ):
            payment = 'PAID CC'
            return payment

def correct_subs_and_tots(row):
    """
    Function is only applicable to cash payments.
    Replace null or zero subtotals with total value, if algebraically sound.
    Replace null or zero totals with subtotal value, if algebraically sound.
    """
    payment = row['Payment Method']
    subtotal = row['Subtotal']
    discount = row['Discount']
    total = row['Total']

    if re.search(no_tax_pat, payment):
        if (pd.notna(total) and total != 0) and (pd.isna(subtotal) or subtotal == 0):
            subtotal = total + discount
            return pd.Series({'Subtotal': subtotal, 'Total': total})
        if (pd.notna(subtotal) and subtotal != 0) and (pd.isna(total) or total == 0):
            total = subtotal - discount
            return pd.Series({'Subtotal': subtotal, 'Total': total})
    return pd.Series({'Subtotal': subtotal, 'Total': total})
                 
# Load data:
o_df = pd.read_json(r'C:\Users\shuju\Desktop\Tannson_Tech_Project\order_details.json')
m_df = pd.read_json(r'C:\Users\shuju\Desktop\Tannson_Tech_Project\monetary_amts.json')

# Create new identifier:
m_df['Transaction Type'] = 'SALE'
voided_transactions_indices = m_df.index[m_df['File Type'] == 'V']
m_df.loc[voided_transactions_indices, 'Transaction Type'] = 'VOID' # Label voided transactions
m_df = m_df.drop(columns='File Type')

# Create new col for missing payment inferences:
m_df['Explicit Payment Method'] = 'YES'
move_col = m_df.pop('Explicit Payment Method')
m_df.insert(2, 'Explicit Payment Method', move_col)

# Correct out-of-bound date:
o_df.loc[5119, 'Date'] = pd.to_datetime('2017-05-24')
m_df.loc[5119, 'Date'] = pd.to_datetime('2017-05-24')

# Correct Subtotal:
m_df['Discount'] = m_df['Discount'].abs()
m_df['Subtotal'] = m_df['Subtotal'] + m_df['Discount']

# Correct negative Total discrepancies:
# Join Price and Service columns from o_df with m_df by indices
temp_df = m_df.join(o_df[['Price', 'Service']])
# Ensure no overlap with voided transactions
temp_df = temp_df.iloc[temp_df.index.difference(voided_transactions_indices)]
# Filter for negatives in Total
negative_tots = temp_df[temp_df['Total'] < 0].copy()
# Create mask for when customers are paid by Tannson
mask_negatives = negative_tots.apply(
    lambda row: paid_customers_fnc(row['Payment Method']) and n_price_coherence_fnc(row['Price']), axis=1
)

# Get indices and define appropriate labels
negative_indices = negative_tots.index[mask_negatives]
m_df.loc[negative_indices, 'Transaction Type'] = 'COMPANY PAID CUSTOMER'
m_df.loc[negative_indices, 'Payment Method'] = 'CASH TO CUSTOMER'
m_df.loc[negative_indices, 'Tax'] = 0
m_df.loc[negative_indices, 'Subtotal'] = m_df.loc[negative_indices, 'Total']
o_df.loc[negative_indices, 'Price'] = 0
o_df.loc[negative_indices, 'Service'] = 'TRADE IN'

# Correct exception cases based on manual inspection of files
m_df.loc[2960, ['Subtotal', 'Total']] = 160
m_df.loc[2960, 'Payment Method'] = 'PAID CASH'
o_df.loc[2960, 'Price'] = 160

m_df.loc[4115, ['Subtotal', 'Total']] = -30
m_df.loc[4115, 'Transaction Type'] = 'COMPANY PAID CUSTOMER'
m_df.loc[4115, 'Payment Method'] = 'CASH TO CUSTOMER'
o_df.loc[4115, 'Price'] = 0
o_df.loc[4115, 'Service'] = 'TRADE IN'

# Correct zero Total discrepancies:
# Filter for zeros in Total
zero_tots = temp_df[temp_df['Total'] == 0].copy()
# Create mask for no repairs
mask_no_repairs = zero_tots.apply(
    lambda row: base_fnc(row['Service'], no_repair_compiler), axis=1
)

# Get indices and define entries as no repairs
no_repair_indices = zero_tots.index[mask_no_repairs]
m_df.loc[no_repair_indices, 'Transaction Type'] = 'VOID'
# o_df.loc[no_repair_indices, 'Service'] = 'DIAGNOSIS - NO REPAIR'

# Create mask for courtesies
mask_courtesies = zero_tots.apply(
    lambda row: base_fnc(row['Service'], courtesy_compiler), axis=1
)
# Get indices and define entries as company courtesies
# Note: Overlap in no_repair_indices checked for accuracy 
courtesy_indices = zero_tots.index[mask_courtesies]
m_df.loc[courtesy_indices, 'Transaction Type'] = 'COMPANY COURTESY'
m_df.loc[courtesy_indices, 'Payment Method'] = None

# Filter for zero totals but relevant prices
w_prices = zero_tots[zero_tots['Price'] != 0].copy()
# Create mask for refunds, defined by abs. value of price == abs. value of refund amt
mask_refunds = w_prices.apply(
    lambda row: (
        abs(
            sum(row['Price']) if isinstance(row['Price'], list) # For list types
            else row['Price'] # For scalars
        )
        == abs(row['Refund Amt'])
    ),
    axis=1
)
# Get indices and void entries
refund_indices = w_prices.index[mask_refunds]
m_df.loc[refund_indices, 'Transaction Type'] = 'VOID'
# Obtain remaining indices and set Price val as the Subtotal and Total val
sum_prices_indices = w_prices.index.difference(refund_indices).to_numpy()
sum_prices = w_prices.loc[sum_prices_indices, 'Price'].apply(
    lambda cell: sum(cell) if isinstance(cell, list) else cell
)
m_df.loc[sum_prices_indices, ['Total', 'Subtotal']] = sum_prices

# Label all remaining zero tots as needing examination
exclude_indices = (
    set(no_repair_indices) | set(courtesy_indices) | set(refund_indices) | set(sum_prices_indices)
)
include_indices = zero_tots.index.difference(exclude_indices).to_numpy()
m_df.loc[include_indices, 'Transaction Type'] = 'FLAGGED — MISSING TOTAL'
m_df.loc[include_indices, 'Explicit Payment Method'] = 'NO'

m_df = m_df.drop('Refund Amt', axis=1)

# Correct Payment Method:
# Set no payment method for voided transactions
m_df.loc[4371, 'Transaction Type'] = 'VOID'
m_df.loc[m_df['Transaction Type'] == 'VOID', 'Payment Method'] = None

sales = m_df[m_df['Transaction Type'].str.contains('SALE')].copy()

# Standardize payment method strings
cash_compiler = re.compile('CASH')
mask_cash = sales.apply(
    lambda row: base_fnc(row['Payment Method'], cash_compiler), axis=1
)
i_cash = sales.index[mask_cash]
m_df.loc[i_cash, 'Payment Method'] = 'PAID CASH'

cc_compiler = re.compile('CC|CREDIT|CARD|VISA|AMEX|AMX|AE')
mask_cc = sales.apply(
    lambda row: base_fnc(row['Payment Method'], cc_compiler), axis=1
)
i_cc = sales.index[mask_cc]
m_df.loc[i_cc, 'Payment Method'] = 'PAID CC'

check_compiler = re.compile('CHECK|CHK')
mask_check = sales.apply(
    lambda row: base_fnc(row['Payment Method'], check_compiler), axis=1
)
i_check = sales.index[mask_check]
m_df.loc[i_check, 'Payment Method'] = 'PAID CHECK'

# Guess missing payment methods
m_inferred_pym = sales[sales['Payment Method'].isna()].copy()
m_inferred_pym['Payment Method'] = m_inferred_pym.apply(infer_payment_fnc, axis=1)

# Manual inspection of remaining files for payment method.
# Transactions 3521, 5676 paid with cash.
# Transaction 4371 voided.
# Payment status of other transactions not mentioned.
m_df.loc[[3521, 5676], 'Payment Method'] = 'PAID CASH'
m_inferred_pym = m_inferred_pym.drop(index=[3521, 5676])
m_inferred_pym.loc[m_inferred_pym['Payment Method'].isna(), 'Payment Method'] = 'UNCLASSIFIED'

# Assign inferences to m_df
m_inferred_pym_indices = m_inferred_pym.index.tolist()
m_df.loc[m_inferred_pym_indices, 'Payment Method'] = m_inferred_pym['Payment Method']
m_df.loc[m_inferred_pym_indices, 'Explicit Payment Method'] = 'NO'

# Guess ambiguous payment methods
a_inferred_pym = m_df[
    ~(m_df['Payment Method'].str.contains('PAID CASH|PAID CC|PAID CHECK|UNCLASSIFIED', na=False)) 
    & (m_df['Transaction Type'].str.contains('SALE', na=False))
    & (m_df['Payment Method'].notna())
    ].copy()
a_inferred_pym['Payment Method'] = a_inferred_pym.apply(infer_payment_fnc, axis=1)
a_inferred_pym.loc[a_inferred_pym['Payment Method'].isna(), 'Payment Method'] = 'UNCLASSIFIED'

# Assign inferences to m_df
a_inferred_pym_indices = a_inferred_pym.index.tolist()
m_df.loc[a_inferred_pym_indices, 'Payment Method'] = a_inferred_pym['Payment Method']
m_df.loc[a_inferred_pym_indices, 'Explicit Payment Method'] = 'NO'

In [2]:
# Correct algebraic discrepancies:

# Create new identifier for sales to define valid transactions in the business setting
m_df.loc[m_df['Transaction Type'].str.contains('SALE', na=False), 'Business Logic Consistency'] = True
# Set unclassified payment methods as being inapplicable for defining logic consistency
m_df.loc[m_df['Payment Method'].str.contains('UNCLASSIFIED', na=False), 'Business Logic Consistency'] = None

# Check for when Subtotal == Total w/o Tax, which violates paying by credit card rule
not_cc_logic = m_df.index[
    (m_df['Payment Method'].str.contains('PAID CC'))
    & (np.isclose
        (m_df['Subtotal'] - m_df['Discount'],
         m_df['Total'], atol=0.02)
      )
    ]
# Set transactions that violate the credit card rule to failing logic consistency
m_df.loc[not_cc_logic, 'Business Logic Consistency'] = False

# Filter for other algebraic discrepancies
tot_mismatches = m_df[
    (m_df['Transaction Type'] == 'SALE')
    & (m_df['Business Logic Consistency'] == True)
    & ~(np.isclose
        (m_df['Subtotal'] - m_df['Discount'] + m_df['Tax'],
         m_df['Total'], atol=0.02)
       )
    ].copy()

# Add Price column
tot_mismatches = tot_mismatches.join(o_df['Price'])

# Create mask where there should be zero tax
mask_zero_tax = tot_mismatches.apply(
    lambda row: (
        (base_fnc(row['Payment Method'], no_tax_compiler)) # Match on CASH or CHECK payments
        or (row['Subtotal'] == row['Total']) # Capture when Subtotal == Total
    ),
    axis=1
)

# Set tax values that match mask_zero_tax conditions to 0
zero_tax_indices = tot_mismatches.index[mask_zero_tax]
m_df.loc[zero_tax_indices, 'Tax'] = 0
tot_mismatches.loc[mask_zero_tax, 'Tax'] = 0

# Compute algebraic discrepancies again (2)
tot_mismatches = tot_mismatches[
    ~np.isclose(
        tot_mismatches['Subtotal']
        - tot_mismatches['Discount']
        + tot_mismatches['Tax'],
        tot_mismatches['Total'],
        atol=0.02
    )
    ]

# Use subtotal as total and total as subtotal if valid
tot_mismatches[['Subtotal', 'Total']] = tot_mismatches.apply(correct_subs_and_tots, axis=1)

# Transactions remain algebraically inconsistent after manual inspection ->
# Set remaining algebraic discrepancies as failing logic consistency
tot_mismatches.loc[
    ~np.isclose(
        tot_mismatches['Subtotal'] 
        - tot_mismatches['Discount'] 
        + tot_mismatches['Tax'], 
        tot_mismatches['Total'], 
        atol=0.02
    ), 'Business Logic Consistency'] = False

# Get indices of previous transactions and reassign altered columns to m_df
fin_mismatches_indices = tot_mismatches.index.tolist()
m_df.loc[fin_mismatches_indices, ['Subtotal', 'Total']] = tot_mismatches[['Subtotal', 'Total']]
m_df.loc[fin_mismatches_indices, 'Business Logic Consistency'] = tot_mismatches['Business Logic Consistency']

In [3]:
def return_item_standardizations(item, item_dict):
    """
    Iterate through item_dict.
    Match an item with a compiler, returning the first standardized name,
    brand, and category of the item.
    """
    for brand, categories in item_dict.items():
        for category, rules in categories.items():
            for standardized_item, compiler in rules:
                if compiler.search(item):
                    return standardized_item, brand, category
    return None, None, None
                    
def standardize_items(items, item_dict, exclude_words):
    """
    Apply name standardization for free text in lists and strings.
    Disregard accessories and other unrelated words captured in the exclude_words list.
    Return standardized item name, brand, and category.
    """
    standardized_items_list = []
    brands_list = []
    categories_list = []
    
    fallback = pd.Series({
        'Items_Standardized': None, 
        'Brand': None, 
        'Category': None
    })

    if items is None:
        pass
        
    if isinstance(items, list):
        items = [item for item in items if not any(re.search(word, item) for word in exclude_words)]

        # Guard against empty lists
        if not items:
            return fallback

        if len(items) > 1:
            for item in items:
                standardized_item, brand, category = return_item_standardizations(item, item_dict)
                standardized_items_list.append(standardized_item)
                brands_list.append(brand)
                categories_list.append(category)
                
            return pd.Series({
                'Items_Standardized': standardized_items_list,
                'Brand': brands_list,
                'Category': categories_list
        })

        else:
            # Guard against one item in list after cleanup
            items = items[0]
            
            standardized_item, brand, category = return_item_standardizations(items, item_dict)
            
            return pd.Series({
                'Items_Standardized': standardized_item,
                'Brand': brand,
                'Category': category
            })
    
    if isinstance(items, str):
        if any(re.search(word, items) for word in exclude_words):
            return fallback
            
        standardized_item, brand, category = return_item_standardizations(items, item_dict)
            
        return pd.Series({
            'Items_Standardized': standardized_item,
            'Brand': brand,
            'Category': category
        })
        
    return fallback

In [17]:
# Standardize Items:

# Remove any iPhone descriptions

# (?!\s*\d) -> Not a #, even with an optional space in-between
# (?=\W|$) -> Either a separator or the end of the string

item_dict = {
    'APPLE' : {
        'PHONE': [
            ('IPHONE 3GS', re.compile(r'\bIPHONE\s*3\s*GS(?=\W|$)')),
            ('IPHONE 3', re.compile(r'\bIPHONE\s*3(?=\W|$)')),
            ('IPHONE 4S', re.compile(r'\bIPHONE\s*4\s*S(?=\W|$)')),
            ('IPHONE 4', re.compile(r'\bIPHONE\s*4(?=\W|$)')),
            ('IPHONE 5C', re.compile(r'\bIPHONE\s*5\s*C(?=\W|$)')),
            ('IPHONE 5S', re.compile(r'\bIPHONE\s*(?:5\s*S|S5)(?=\W|$)')),
            ('IPHONE 5', re.compile(r'\bIPHONE\s*5(?=\W|$)')),
            ('IPHONE 6 PLUS', re.compile(r'\bIPHONE\s*6\s*(?:\+|PLUSS*)(?=\W|$)')),
            ('IPHONE 6S PLUS', re.compile(r'\bIPHONE\s*6\s*S\s*(?:\+|PLUSS*)(?=\W|$)')),
            ('IPHONE 6S', re.compile(r'\bIPHONE\s*6\s*S(?=\W|$)')),
            ('IPHONE 6', re.compile(r'\bIPHONE\s*6(?=\W|$)')),
            ('IPHONE 7 PLUS', re.compile(r'\bIPHONE\s*7\s*(?:\+|PLUSS*)(?=\W|$)')),
            ('IPHONE 7', re.compile(r'\bIPHONE\s*7(?=\W|$)')),
            ('IPHONE 8 PLUS', re.compile(r'\bIPHONE\s*8\s*(?:\+|PLUSS*)(?=\W|$)')),
            ('IPHONE 8', re.compile(r'\bIPHONE\s*8(?=\W|$)')),
            ('IPHONE 11 PRO MAX', re.compile(r'\bIPHONE\s*11\s*PRO\s*MAX(?=\W|$)')),
            ('IPHONE 11 PRO', re.compile(r'\bIPHONE\s*11\s*PRO(?=\W|$)')),
            ('IPHONE 11', re.compile(r'\bIPHONE\s*11(?=\W|$)')),
            ('IPHONE 12 PRO MAX', re.compile(r'\bIPHONE\s*12\s*PRO\s*MAX(?=\W|$)')),
            ('IPHONE 12 PRO', re.compile(r'\bIPHONE\s*12\s*PRO(?=\W|$)')),
            ('IPHONE 12 MINI', re.compile(r'\bIPHONE\s*12\s*MINI(?=\W|$)')),
            ('IPHONE 12', re.compile(r'\bIPHONE\s*12(?=\W|$)')),
            ('IPHONE 13 PRO MAX', re.compile(r'\bIPHONE\s*13\s*PRO\s*MAX\b')),
            ('IPHONE 13 PRO', re.compile(r'\bIPHONE\s*13\s*PRO\b')),
            ('IPHONE 13', re.compile(r'\bIPHONE\s*13(?=\W|$)')),
            ('IPHONE 14 PLUS', re.compile(r'\bIPHONE\s*14\s*(?:\+|PLUSS*)(?=\W|$)')),
            ('IPHONE 14 PRO MAX', re.compile(r'\bIPHONE\s*14\s*PRO\s*MAX\b')),
            ('IPHONE 14 PRO', re.compile(r'\bIPHONE\s*14\s*PRO\b')),
            ('IPHONE 14', re.compile(r'\bIPHONE\s*14(?=\W|$)')),
            ('IPHONE 15 PLUS', re.compile(r'\bIPHONE\s*15\s*(?:\+|PLUSS*)(?=\W|$)')),
            ('IPHONE 15 PRO MAX', re.compile(r'\bIPHONE\s*15\s*PRO\s*MAX\b')),
            ('IPHONE 15 PRO', re.compile(r'\bIPHONE\s*15\s*PRO\b')),
            ('IPHONE 15', re.compile(r'\bIPHONE\s*15(?=\W|$)')),
            ('IPHONE 16 PLUS', re.compile(r'\bIPHONE\s*16\s*(?:\+|PLUSS*)(?=\W|$)')),
            ('IPHONE 16 PRO MAX', re.compile(r'\bIPHONE\s*16\s*PRO\s*MAX\b')),
            ('IPHONE 16 PRO', re.compile(r'\bIPHONE\s*16\s*PRO\b')),
            ('IPHONE 16', re.compile(r'\bIPHONE\s*16(?=\W|$)')),
            ('IPHONE SE', re.compile(r'\bIPHONE\s*(?:SE|5\s*SE|8\s*SE|SE2)(?=\W|$)')),
            ('IPHONE XR', re.compile(r'\bIPHONE\s*XR\b')),
            ('IPHONE XS MAX', re.compile(r'\bIPHONE\s*XS\s*MAX\b')),
            ('IPHONE XS', re.compile(r'\bIPHONE\s*XS\b')),
            ('IPHONE X', re.compile(r'\bIPHONE\s*X\b')),
            ('IPHONE MODEL', re.compile(r'\bIPHONE\b'))
        ],

        'COMPUTER': [
            ('IMAC', re.compile(r'\bIMAC\b')),
            ('MACBOOK PRO', re.compile(r'\bMAC\s*BOOK\s*PRO\b')),
            ('MACBOOK AIR', re.compile(r'\bMAC\s*BOOK\s*AIR\b')),
            ('MACBOOK', re.compile(r'\bMAC\s*BOOK\b')),
            ('MAC PRO', re.compile(r'\bMAC\s*(?:PRO\s*)?(?:TOWER|PRO)\b')),
            ('UNCLASSIFIED', re.compile(r'\bMAC\b'))
        ],

        'TABLET': [
            ('IPAD (2ND GEN)', re.compile(r'\bIPAD\s*2(?:ND)?(?=\W|$)')),
            ('IPAD (4TH GEN)', re.compile(r'\bIPAD\s*4(?:TH)?(?=\W|$)')),
            ('IPAD (5TH GEN)', re.compile(r'\bIPAD\s*5(?:TH)?(?=\W|$)')),
            ('IPAD (6TH GEN)', re.compile(r'\bIPAD\s*6(?:TH)?(?=\W|$)')),
            ('IPAD (7TH GEN)', re.compile(r'\bIPAD\s*7(?:TH)?(?=\W|$)')),
            ('IPAD (8TH GEN)', re.compile(r'\bIPAD\s*8(?:TH)?(?=\W|$)')),
            ('IPAD (9TH GEN)', re.compile(r'\bIPAD\s*9(?:TH)?(?=\W|$)')),
            ('IPAD (10TH GEN)', re.compile(r'\bIPAD\s*10(?:TH)?(?=\W|$)')),
            ('IPAD AIR', re.compile(r'\bIPAD\s*AIR(?=\W|$)')),
            ('IPAD PRO', re.compile(r'\bIPAD\s*PRO(?=\W|$)')),
            ('IPAD MINI', re.compile(r'\bIPAD\s*MINI(?=\W|$)')),
            ('IPAD MODEL', re.compile(r'\bIPAD\b')),
        ],

        'MEDIA PLAYER': [
            ('IPOD CLASSIC', re.compile(r'\bIPOD\s*CLASSIC(?=\W|$)')),
            ('IPOD TOUCH', re.compile(r'\bIPOD\s*TOUCH(?=\W|$)')),
            ('IPOD MODEL', re.compile(r'\bIPOD\b')),
        ],

        'OTHER': [
            ('WATCH', re.compile(r'\b(?:APPLE\s*WATCH|IWATCH)(?=\W|$)'))
        ],
    },

    'SAMSUNG': {
        'PHONE': [
            ('GALAXY A', re.compile(r'\b(?:SAMSUNG?\s*GALAXY|SAMSUNG|GALAXY)\s*A\d{1,3}\w*\b')),
            ('GALAXY J', re.compile(r'\b(?:SAMSUNG\s*GALAXY|SAMSUNG|GALAXY)\s*(?:SM-?J|J)\d{1,3}\w*\b')),
            ('GALAXY S', re.compile(r'\b(?:SAMSUNG\s*GALAXY|SAMSUNG|GALAXY)\b.*\b(?:(SM-?)?(?:G9|S9)|I|S)(?:\d{1,3})?(?:\+|\w*)\b')),
            ('GALAXY CORE', re.compile(r'\b(?:SAMSUNG\s*GALAXY|SAMSUNG|GALAXY)\s*(?:SM-?G36|CORE)\w*\b')),
            ('GALAXY GRAND', re.compile(r'\b(?:SAMSUNG\s*GALAXY|SAMSUNG|GALAXY)\s*(?:SM-?G53|GRAND)\w*\b')),
            ('GALAXY NOTE', re.compile(r'\b(?:SAMSUNG\s*GALAXY|SAMSUNG|GALAXY)\s*(?:SM-?N|N|NOTE)\s*\d{1,4}(?:\+|\w*)\b')),
            ('GALAXY Z FLIP', re.compile(r'\b(?:SAMSUNG\s*GALAXY|SAMSUNG|GALAXY)\s*Z?\s*FLIP\s*\d?\b')),
            ('GALAZY Z FOLD', re.compile(r'\b(?:SAMSUNG\s*GALAXY|SAMSUNG|GALAXY)\s*Z?\s*FOLD\s*\d?\b')),
            ('GALAXY MEGA', re.compile(r'\b(?:SAMSUNG\s*GALAXY|SAMSUNG|GALAXY)\s*MEGA\b')),
            ('GALAXY EXPRESS', re.compile(r'\b(?:SAMSUNG\s*GALAXY|SAMSUNG|GALAXY)\s*EXPRESS\b')),
            ('GALAXY ON', re.compile(r'\b(?:SAMSUNG\s*GALAXY|SAMSUNG|GALAXY)\b.*\bON\s*\d{1}\b')),
            ('GALAXY XCOVER', re.compile(r'\b(?:SAMSUNG\s*GALAXY|SAMSUNG|GALAXY)\s*XCOVER\b')),
            ('INTENSITY', re.compile(r'\b(?:SAMSUNG\s*GALAXY|SAMSUNG|GALAXY)\s*(?:SCH-?U|U)\d{3}\b')),
            ('UNCLASSIFIED', re.compile(r'\bSAMSUNG\s*PHONE\b'))
        ],

        'TABLET': [
            ('GALAXY TAB', re.compile(r"""
            \b(?:SAMSUNG\s*GALAXY|SAMSUNG|GALAXY)\s*
            (?:
                SM-?(?:T|P)\d{3,4}\w*
                |T-?\d{3}\w*
                |TAB\s*\w*
                |GT-?N
                |TABLET
                )\w*\b""", re.VERBOSE)),
        ],

        'COMPUTER': [
            ('LAPTOP', re.compile(r'\bSAMSUNG\s*(?:CHROME(?:\s*BOOK)?|LAPTOP|NOTEBOOK|CHRONOS|NP\d{3}\w*)\b')),
        ],

        'UNCLASSIFIED': [
            ('PRODUCT', re.compile(r'\bSAMSUNG\b'))
        ]
    },

    'ALCATEL': {
        'PHONE': [
            ('A30', re.compile(r'\bALCATEL\s*A\d{1,3}\b')),
        ]
    },

    'MICROSOFT': {
        'TABLET': [
            ('SURFACE', re.compile(r'\bMICROSOFT\s*SURFACE\s*(?:PRO|\d{1})\b')),
        ],
        
        'UNCLASSIFIED': [
            ('SURFACE', re.compile(r'\bMICROSOFT\s*SURFACE\b'))
        ]
    },
    
    'GOOGLE': {
        'PHONE': [
            ('PIXEL', re.compile(r'\bGOOGLE\s*PIXEL(?:\s*\d{1}\w*)?\b')),
            ('NEXUS', re.compile(r'\bGOOGLE\s*NEXUS\b')),
        ],
    },

    'NOKIA': {
        'PHONE': [
            ('LUMIA', re.compile(r'\bNOKIA\b.*\bLUMIA\b')),
            ('X SERIES', re.compile(r'\bNOKIA\s*X(?:\d{3})?\b'))
        ]
    },

    'ASUS': {
        'TABLET': [
            ('TRANSFORMER', re.compile(r'\bASUS\b.*\bTF\s*\d{3}\b')),
            ('NEXUS', re.compile(r'\bASUS\b.*\bNEXUS\b')),
            ('ZENPAD', re.compile(r'\bASUS\b.*\bZENPAD\b')),
            ('UNCLASSIFIED', re.compile(r'\bASUS\s*TABLET\b'))
        ],
        
        'COMPUTER': [
            ('LAPTOP', re.compile(r'\bASUS\b.*\b(?:LAPTOP|(?:NET|NOTE|CHROME|VIVO|ZEN)BOOK?)\b')),
            ('DESKTOP', re.compile(r'\bASUS\b.*\b(?:PC|ALL[\s\-]?IN[\s\-]?ONE)\b')),
        ],

        'PHONE': [
            ('ZENFONE', re.compile(r'\bASUS\b.*\b(?:PHONE|ZENFONE|ZENPHONE)\b'))
        ],

        'OTHER': [
            ('MOTHERBOARD', re.compile(r'\bASUS\b.*\bMOTHERBOARD')),
        ],
        
        'UNCLASSIFIED': [
            ('PRODUCT', re.compile(r'\bASUS\b'))
        ]
    },

    'CORE INNOVATION': {
        'COMPUTER': [
            ('LAPTOP', re.compile(r'\bCORE\s*INNOVATION\b.*\bLAPTOP\b')),
        ],
    },

    'DELL': {
        'COMPUTER': [
            ('LAPTOP', re.compile(r'\bDELL\b.*\b(?:NOTEBOOK|LAPTOP)\b')),
            ('DESKTOP', re.compile(r'\bDELL\b.*\bDESKTOP\b'))
        ],
        
        'UNCLASSIFIED': [
            ('PRODUCT', re.compile(r'\bDELL\b')),
        ],
    },
    
    'HP': {
        'OTHER': [
            ('PRINTER', re.compile(r'\bHP(?:\s*PRINTER|\s*ENVY\s*PHOTO)'))
        ],
        
        'COMPUTER': [
            ('DESKTOP', re.compile(r"""
            \bHP\b
            (?:
                \b.*\bDESKTOP\b
                |\b.*\bPC
                |\b.*\bALL(?:\s*|-)IN(?:\s*|-)ONE\b
                |\b.*\bTOWER\b
                |\s*SLIMLINE
                |\s*ENVY\s*2\d
                |\s*PAVILION\s*2\d
                |\s*2\d{1}\b
                |\s*110
                )""", re.VERBOSE)),
            ('LAPTOP', re.compile(r"""
             \bHP
            (?:
                \b.*\bLAPTOP\b
                |\b.*\bTOUCH
                |\s*CHROMEBOOK\b
                |\s*NOTEBOOK\b
                |\s*PROBOOK\b
                |\s*ELITEBOOK\b
                |\s*STREAM\b
                |\s*SPECTRE\b
                |\s*DV
                |\s*ENVY\s*(?:1\d|X\d{3}|DM|M)
                |\s*PAVILION\s*(?:1\d|X\d{3}|DM|M)
                |\s*\d{1,4}(?:-\w+)?
                |\s*G\d{2}
                )""", re.VERBOSE)),
            ('UNCLASSIFIED', re.compile(r'\bHP\s*(?:COMPUTER|PAVILION|ENVY)\b')),
        ],
        
        'UNCLASSIFIED': [
            ('PRODUCT', re.compile(r'\bHP\b')),
        ],
    },

    'LG': {
        'PHONE': [
            ('FORTUNE', re.compile(r'\bLG\s*FORTUNE\b')),
            ('REVERE', re.compile(r'\bLG\s*REVERE\b')),
            ('VELVET', re.compile(r'\bLG\s*VELVET\b')),
            ('G SERIES', re.compile(r'\bLG\b.*\bG\d{1,2}\b')),
            ('K SERIES', re.compile(r'\bLG\s*K\d{1,3}\b')),
            ('V SERIES', re.compile(r'\bLG\s*V\d{2}\b')),
            ('Q SERIES', re.compile(r'\bLG\s*Q\d')),
            ('STYLO', re.compile(r'\bLG\s*(?:STYLO|STYLUS)\b.*\b\d{1,2}\b')),
            ('UNCLASSIFIED', re.compile(r'\bLG[-\s]*(?:LS|MS|VS|VN|VX|H|PHONE)\s*(?:\d{3,4})*\w*\b'))
        ],

        'TABLET': [
            ('G PAD', re.compile(r'\bLG\s*G\s*PAD\b')),
            ('UNCLASSIFIED', re.compile(r'\bLG[-\s]*VK\s*\d{3,4}\w*\b'))
        ],

        'COMPUTER': [
            ('LAPTOP', re.compile(r'\bLG\s*(?:LAPTOP|GRAM)\b'))
        ],

        'UNCLASSIFIED': [
            ('PRODUCT', re.compile(r'\bLG\b'))
        ]
    },

    'TOSHIBA': {
        'COMPUTER': [
            ('LAPTOP', re.compile(r"""
            \bTOSHIBA\s*
                (?:
                    LAPTOP\b
                    |NOTEBOOK\b
                    |SAT[A-Z]LLITE\b
                    |C(?:\d)?55
                    |L\d{2,3}
                    |S\d{2,3}
                    )""", re.VERBOSE))
        ],

        'TABLET': [
            ('EXCITE', re.compile(r'\bTOSHIBA\s*EXCITE\s*(?:TABLET)?\b'))
        ],
        
        'UNCLASSIFIED': [
            ('PRODUCT', re.compile(r'\bTOSHIBA\b'))
        ]
    },

    'LENOVO': {
        'COMPUTER': [
            ('LAPTOP', re.compile(r"""
            \bLENOVO\s*
            (?:
                LAPTOP\b
                |CHROMEBOOK\b
                |FLEX\b
                |IDEAPAD\b
                |THINKPAD\b
                |T\d{3}\b
                |YOGA\b
                |LEGION\s*(?!T)\w+\b
                |X1\b
                |G\d
                |V\d{2})
                """, re.VERBOSE)),
            ('DESKTOP', re.compile(r"""
            \bLENOVO\s*
            (?:
                DESKTOP\b
                |PC\b
                |\b.*\bALL(?:\s*|-)IN(?:\s*|-)ONE\b
                |S\d{3}\b)
                """, re.VERBOSE))
        ],

        'TABLET': [
            ('TAB', re.compile(r'\bLENOVO\s*(?:TAB\s*(?:A|M)\d{1,2})')),
            ('UNCLASSIFIED', re.compile(r'\bLENOVO\s*TABLET\b'))
        ],

        'UNCLASSIFIED': [
            ('PRODUCT', re.compile(r'\bLENOVO\b'))
        ],
    },

    'ACER': {
        'UNCLASSIFIED': [
            ('PRODUCT', re.compile(r'\bACER\b'))
        ]
    },

    'MOTOROLA': {
        'PHONE': [
            ('MOTO G5', re.compile(r'\bMOTOROLA\s*(?:MOTO\s*)?G5(?=\W|$)')),
        ]
    },

    'EPIK': {
        'COMPUTER': [
            ('LAPTOP', re.compile(r'\bEPIK\b.*\bLAPTOP\b')),
        ],
    },

    'BLU': {
        'UNCLASSIFIED': [
            ('PRODUCT', re.compile(r'\bBLU\b'))
        ]
    },

    'SONY ERICSSON': {
        'UNCLASSIFIED': [
            ('PRODUCT', re.compile(r'\bSONY\s*ERICSSON\b'))
        ]
    }     
}

exclude_words = [
    'GLASS', 'CUSTOMIZE', 'RETURN', 'TRADE', 'PROBLEM',
    'PROTECTOR', 'SIM TRAY', 'KEEP', 'BATH', 'RETURN', 'PAID',
    'BAND', 'PROBLEM', 'DEMO', 'PICKUP', 'RESTORING', '@',
    '\bCHROME\b', 'SCAN', 'CUSTOMER', 'RECYCLE', 'FAILED',
    'CORRUPTED', 'CRASH', 'SIGNAL', 'FROZE', 'SETUP', '\bBAR\b'
    'ADJUST', 'DISCOUNT', 'DISABLE'
]

In [28]:
o_df_copy = o_df.copy()
o_df_copy[['Items_Standardized', 'Brand', 'Category']] = 'TEST'
o_df_copy[['Items_Standardized', 'Brand', 'Category']] = o_df_copy['Item'].apply(lambda row: standardize_items(row, item_dict, exclude_words))
na = o_df_copy[o_df_copy['Items_Standardized'].isna()]
na = na.astype(str)
na[na['Item'].str.contains('LENOVO', na=False)].info()
na[na['Item'].str.contains('LENOVO', na=False)]
o_df_copy = o_df_copy.astype(str)
o_df_copy[o_df_copy['Item'].str.contains('SAMSUNG', na=False)]

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, 756 to 2729
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Date                4 non-null      object
 1   Item                4 non-null      object
 2   Service             4 non-null      object
 3   Price               4 non-null      object
 4   Items_Standardized  4 non-null      object
 5   Brand               4 non-null      object
 6   Category            4 non-null      object
dtypes: object(7)
memory usage: 256.0+ bytes


,Date,Item,Service,Price,Items_Standardized,Brand,Category
24,2015-12-29 00:00:00,SAMSUNG TAB GT-P5210,REPLACE CHARGING PORT,60.0,GALAXY TAB,SAMSUNG,TABLET
69,2024-07-27 00:00:00,SAMSUNG A15,TRANSFER DATA FROM OLD DEVICE,80.0,GALAXY A,SAMSUNG,PHONE
88,2025-10-29 00:00:00,SAMSUNG FLIP,CLEAN CHARGING PORT,80.0,GALAXY Z FLIP,SAMSUNG,PHONE
121,2023-03-30 00:00:00,SAMSUNG A536U,SCREEN REPLACEMENT,200.0,GALAXY A,SAMSUNG,PHONE
126,2016-08-19 00:00:00,"['SONY VAIO NOTEBOOK', 'SAMSUNG EVO SSD 500GB']",REPLACE HD AND REINSTALL WINDOWS 7,100.0,GALAXY S,SAMSUNG,PHONE
127,2023-01-28 00:00:00,SAMSUNG S6,REPLACE BATTERY,75.0,GALAXY S,SAMSUNG,PHONE
128,2022-11-21 00:00:00,SAMSUNG S8,REPLACE BATTERY,75.0,GALAXY S,SAMSUNG,PHONE
134,2018-11-01 00:00:00,SAMSUNG S5 BATTERY,OTHER,32.83,GALAXY S,SAMSUNG,PHONE
144,2018-04-26 00:00:00,SAMSUNG S7 EDGE BLUE CORAL,SCREEN REPLACEMENT,300.0,GALAXY S,SAMSUNG,PHONE
150,2021-07-12 00:00:00,SAMSUNG A02S,REMOVE GOOGLE ACCT.,60.0,GALAXY A,SAMSUNG,PHONE


In [29]:
o_df = o_df.astype(str)
q = o_df[o_df['Item'].str.contains('MICROSOFT')]
q.groupby('Item').size()

Item
MICROSOFT NOKIA LUMIA 950                                        1
MICROSOFT SURFACE                                                3
MICROSOFT SURFACE 2013                                           1
MICROSOFT SURFACE 4 (1724)                                       1
MICROSOFT SURFACE 7                                              1
MICROSOFT SURFACE PRO 4 (2964)                                   1
MICROSOFT SURFACE PRO 7                                          1
PROBLEM: ERROR POPUP WHEN CLOSE MICROSOFT                        1
['DELL SYSTEM (DIKACHI1999)', 'MICROSOFT SURFACE 3 (2003)']      1
['HP LAPTOP', 'PROBLEM: OPEN TO MICROSOFT POWERSHELL, SLOW,']    1
['IPAD 7TH', 'MICROSOFT SURFACE', 'IPHONE SE']                   1
['LENOVO LAPTOP', 'PROBLEM: STUCK MICROSOFT OFFICE']             1
['MACBOOK (1997/1994)', 'MICROSOFT SURFACE (OSPREY321/123)']     1
dtype: int64

12/31/2025:
* 332 of 421 zero totals fixed. **Remaining 89** requires manual inspection or inquiry to client.

01/01/2026:
* 991 of 1076 algebraic discrepancies in totals fixed. Remaining: 85.

01/03/2026:
* Created 'Explicit Payment Type' and identified VOIDs. Tomorrow: Infer missing payment method fnc (CASH if zero or null tax and CC if tax), get indices of inferences, set EPM to 'NO'. Note: Example of reconstructing variables from noisy observational data.

01/04/2026:
* Created CASH/CHECK or CC inference fnc. Standardized payment method strings. Tomorrow: 1) Incorporate DIAGNOSTIC and COURTESY key words found in Payment column to mask_no_repairs and mask_courtesies. 2) Standardize strings in lists (if list contains CASH, return CASH as the sole value instead of the list — might not need to create a new fnc.)

01/06/2026:
* Classified payment methods of all sales. Tomorrow: Correct all remaining entries. Continue dealing with algebraic discrepancies.

01/08/2026:
* Classified refunds as VOID. Set Price col value or sum of Price col values as Total. Tomorrow: Continue dealing with algebraic discrepancies.

01/11/2026:
* 1071 of 1078 algebraic discrepancies in totals fixed. **Remaining: 7**. All other entries are algebraically unsound even with manual inspection, so mark as violating business logic consistency. Tomorrow: Standardize Items.

01/18/2026:
* Standardized: ASUS, APPLE, SAMSUNG.

01/19/2026:
* Created function to apply item name standardizations.

01/20/2026:
* Standardized: GOOGLE, MICROSOFT, NOKIA, LG.

01/21/2026:
* Standardized: TOSHIBA, HP, LENOVO.

01/21/2026:
* Standardized: DELL, SONY, ACER, AMAZON, HUAWEI.